In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

In [33]:
train_images_dir = '/Users/admin/Downloads/data/training_images'
annotations_path = os.path.join(train_images_dir, '/Users/admin/Downloads/data/train_solution_bounding_boxes (1).csv')

In [34]:
annotations_df = pd.read_csv(annotations_path)

In [39]:
def load_images_and_annotations(image_dir, annotations_df):
    images = []
    labels = []

    for idx, row in annotations_df.iterrows():
        img_path = os.path.join(image_dir, row['image'])
        img = cv2.imread(img_path)
        
        # Check if the image is loaded successfully
        if img is None:
            print(f"Error: Failed to load image {img_path}")
            continue
        
        # Resize and normalize the image
        img = cv2.resize(img, (224, 224))  # Resize image to fit ResNet50 input size
        img = img / 255.0  # Normalize pixel values
        images.append(img)
        
        # Extract bounding box coordinates
        label = [row['xmin'], row['ymin'], row['xmax'], row['ymax']]
        labels.append(label)

    return np.array(images), np.array(labels)

# Load images and annotations
train_images, train_labels = load_images_and_annotations(train_images_dir, annotations_df)


In [40]:
# Load images and annotations
train_images, train_labels = load_images_and_annotations(train_images_dir, annotations_df)

# Split data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)


In [41]:
# Step 2: Build the Model

# Build model
def build_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False
    
    model = Sequential([
        base_model,
        Flatten(),
        Dense(512, activation='relu'),
        Dense(4)  # 4 output nodes for bounding box coordinates (x1, y1, x2, y2)
    ])
    return model

model = build_model()
model.summary()


94765736/94765736 [==============================] - 8s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 512)               51380736  
                                                                 
 dense_1 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 74970500 (285.99 MB)
Trainable params: 51382788 (196.01 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [42]:

# Step 3: Train the Model with Different Optimizers

# Define optimizers
optimizers = {
    'SGD': SGD(lr=0.001),
    'Adam': Adam(),
    'RMSprop': RMSprop()
}

# Train model with different optimizers
results = {}
for optimizer_name, optimizer in optimizers.items():
    print(f"Training with optimizer: {optimizer_name}")
    model = build_model()
    model.compile(optimizer=optimizer, loss='mse')  # Mean Squared Error loss for bounding box regression
    history = model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))
    results[optimizer_name] = history

Training with optimizer: SGD
Epoch 1/10
14/14 [==============================] - 73s 5s/step - loss: nan - val_loss: nan
Epoch 2/10
14/14 [==============================] - 70s 5s/step - loss: nan - val_loss: nan
Epoch 3/10
14/14 [==============================] - 70s 5s/step - loss: nan - val_loss: nan
Epoch 4/10
14/14 [==============================] - 81s 6s/step - loss: nan - val_loss: nan
Epoch 5/10
14/14 [==============================] - 86s 6s/step - loss: nan - val_loss: nan
Epoch 6/10
14/14 [==============================] - 74s 5s/step - loss: nan - val_loss: nan
Epoch 7/10
14/14 [==============================] - 71s 5s/step - loss: nan - val_loss: nan
Epoch 8/10
14/14 [==============================] - 86s 6s/step - loss: nan - val_loss: nan
Epoch 9/10
14/14 [==============================] - 117s 8s/step - loss: nan - val_loss: nan
Epoch 10/10
14/14 [==============================] - 108s 8s/step - loss: nan - val_loss: nan
Training with optimizer: Adam
Epoch 1/10
14/14 [

In [45]:
# Evaluate and compare results
for optimizer_name, history in results.items():
    print(f"\nOptimizer: {optimizer_name}")
    print("Final Training Loss:", history.history['loss'][-1])
    print("Final Validation Loss:", history.history['val_loss'][-1])


Optimizer: SGD
Final Training Loss: nan
Final Validation Loss: nan

Optimizer: Adam
Final Training Loss: 22132.759765625
Final Validation Loss: 19982.908203125

Optimizer: RMSprop
Final Training Loss: 22618.35546875
Final Validation Loss: 19447.142578125
